In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from tqdm import trange
import sys

from models.tfkeras_resnets import build_single_block_resnet, build_resnet
from dataset_utils.tf_dataset_preprocessors_image_classification import UnpackImagesLabels, ConvertLabelsToOneHot, DecodeImages, DecodeJPEGImages, ResizeWithPad, RandomFlipLeftRight, RandomCrop, Resize
from dataset_utils.tfrecord_dataset_creator import TFRecordDatasetCreator, get_tfrecord_paths
from training.training_v2 import Training

from tensorflow.train import AdamOptimizer

print("TensorFlow version: {}".format(tf.__version__))

%matplotlib inline

TensorFlow version: 1.12.0


In [2]:
num_classes = 257
image_size = (224, 224, 3)

blocks_resnet50  = [3, 4,  6, 3]
blocks_resnet101 = [3, 4, 23, 3]
blocks_resnet152 = [3, 8, 36, 3]

In [3]:
tf.keras.backend.clear_session()

resnet = build_resnet(image_size=image_size,
                        kernel_type='regular',#'antisymmetric',
                        include_top=True,
                        fc_activation=None,
                        num_classes=num_classes,
                        l2_regularization=0.0,
                        subtract_mean=127.5,
                        divide_by_stddev=127.5,
                        version=1,
                        preset=None,#'resnet50',
                        blocks_per_stage=[2, 2, 2, 2],
                        filters_per_block=[[64, 64, 128],
                                           [64, 64, 256],
                                           [128, 128, 512],
                                           [256, 256, 1024]],
                        use_batch_norm=False)

In [4]:
train_directory = '/pierluigiferrari/datasets/Caltech256/train/'
val_directory = '/pierluigiferrari/datasets/Caltech256/val/'

feature_schema = {'image': tf.FixedLenFeature([], tf.string),
                  'filename': tf.FixedLenFeature([], tf.string),
                  'label': tf.FixedLenFeature([], tf.int64)}

unpack_images_labels = UnpackImagesLabels()
decode_images = DecodeJPEGImages(channels=3)
random_crop = RandomCrop(aspect_ratio=1,
                         scale=1.0,
                         channels=3,
                         num_parallel_calls=None)
resize = Resize(target_size=image_size[:2])
resize_with_pad = ResizeWithPad(target_size=image_size[:2])
random_flip_left_right = RandomFlipLeftRight()
convert_labels_to_one_hot = ConvertLabelsToOneHot(num_classes=257)

train_preprocessors = [unpack_images_labels,
                       decode_images,
                       random_flip_left_right,
                       resize_with_pad,
                       convert_labels_to_one_hot]

val_preprocessors = [unpack_images_labels,
                     decode_images,
                     resize_with_pad,
                     convert_labels_to_one_hot]

train_dataset_creator = TFRecordDatasetCreator(tfrecord_paths=get_tfrecord_paths(directory=train_directory, extension='tfrecord'),
                                               feature_schema=feature_schema,
                                               batch_size=32,
                                               preprocessors=train_preprocessors,
                                               repeat=True,
                                               num_epochs=None,
                                               shuffle=True,
                                               shuffle_buffer_size=10000,
                                               num_parallel_reads=None,
                                               num_parallel_calls=None)

val_dataset_creator = TFRecordDatasetCreator(tfrecord_paths=get_tfrecord_paths(directory=val_directory, extension='tfrecord'),
                                             feature_schema=feature_schema,
                                             batch_size=32,
                                             preprocessors=val_preprocessors,
                                             repeat=True,
                                             num_epochs=None,
                                             shuffle=False,
                                             shuffle_buffer_size=10000,
                                             num_parallel_reads=None,
                                             num_parallel_calls=None)

train_dataset = train_dataset_creator.create_dataset()
val_dataset = val_dataset_creator.create_dataset()

In [5]:
iterator = train_dataset.make_one_shot_iterator()
features, labels = iterator.get_next()

model_output = resnet(features)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf.stop_gradient(labels), logits=model_output))

train_op = tf.train.AdamOptimizer().minimize(loss)

softmax_output = tf.nn.softmax(model_output, name='softmax_output')
predictions_argmax = tf.argmax(softmax_output, axis=-1, name='predictions_argmax', output_type=tf.int64)
labels_argmax = tf.argmax(labels, axis=-1, name='labels_argmax', output_type=tf.int64)

mean_loss_value, mean_loss_update_op = tf.metrics.mean(loss)
acc_value, acc_update_op = tf.metrics.accuracy(labels=labels_argmax, predictions=predictions_argmax)

In [6]:
epochs = 3
steps_per_epoch = 1000

In [7]:
with tf.Session() as sess:

    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    for epoch in range(1, epochs+1):

        tr = trange(steps_per_epoch, file=sys.stdout)
        tr.set_description('Epoch {}/{}'.format(epoch, epochs))

        for train_step in tr:

            _a, _b, mean_loss, _c, accuracy = sess.run([train_op,
                                                        mean_loss_update_op,
                                                        mean_loss_value,
                                                        acc_update_op,
                                                        acc_value])

            tr.set_postfix(ordered_dict={'loss': mean_loss,
                                         'accuracy': accuracy})

Epoch 3/3: 100%|██████████| 1000/1000 [02:48<00:00,  6.52it/s, loss=4.93, accuracy=0.0954] 


In [5]:
resnet.compile(optimizer='adam',
               loss='categorical_crossentropy',
               metrics=['accuracy'])

history = resnet.fit(train_dataset,
                     epochs=epochs,
                     steps_per_epoch=steps_per_epoch)

Epoch 1/5
1000/1000 [==============================] - 274s 274ms/step - loss: 4.7559 - acc: 0.0989 - val_loss: 4.4607 - val_acc: 0.1230
Epoch 2/5
1000/1000 [==============================] - 241s 241ms/step - loss: 3.9325 - acc: 0.1750 - val_loss: 3.8046 - val_acc: 0.1926
Epoch 3/5
1000/1000 [==============================] - 262s 262ms/step - loss: 3.4028 - acc: 0.2457 - val_loss: 3.4681 - val_acc: 0.2456
Epoch 4/5
1000/1000 [==============================] - 256s 256ms/step - loss: 2.9319 - acc: 0.3213 - val_loss: 3.1591 - val_acc: 0.3051
Epoch 5/5
240/240 [==============================] - 33s 139ms/step


In [6]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()

x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

print(x_train.shape)
print(y_train[0])

(60000, 28, 28, 1)
5
